In [1]:
import pandas as pd
from indicators import RSI, extract_bb, MFI, CMF
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import warnings
import torch.nn as nn

from IPython.display import clear_output
import plotly.graph_objects as go
import plotly.express as px
from sb3_contrib import RecurrentPPO
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import ProgressBarCallback

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("gzpn_data.csv")
df = df.dropna().reset_index(drop=True)  # .sample(frac=1)
df = df.drop(columns=["<TICKER>", "<PER>", "<DATE>", "<TIME>"])
df.columns = ["open", "high", "low", "close", "volume"]
df

,open,high,low,close,volume
0,162.65,162.70,162.43,162.49,218280
1,162.49,162.50,162.31,162.42,174320
2,162.43,162.49,162.30,162.43,128060
3,162.45,162.46,162.28,162.29,125240
4,162.30,162.37,162.20,162.37,228440
...,...,...,...,...,...
2127,163.37,163.40,163.30,163.40,171700
2128,163.40,163.82,163.39,163.71,623050
2129,163.70,163.82,163.61,163.80,314660
2130,163.80,163.82,163.60,163.70,136580


In [42]:
n_steps = 14

prices = df["close"]

rsi_values = RSI(prices=prices, n_steps=n_steps)
bb_values = extract_bb(prices=prices, n_steps=n_steps)
mfi_values = MFI(
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    volume=df["volume"],
    n_steps=n_steps,
)


cmf_values = CMF(
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    volume=df["volume"],
    n_steps=n_steps,
)

assert len(rsi_values) == len(
    bb_values
), f"Indicators length don't coincide: {len(rsi_values)} and {len(bb_values)}"

In [43]:
def prepare_target(df, steps_obs: int = 3):
    targets = []
    for i in range(0, len(df) - steps_obs):
        current_price = df["close"].iloc[i]
        max_price = df["high"].iloc[i + 1 : i + 1 + steps_obs].max()
        targets.append(max_price > current_price)
    targets += [0] * steps_obs
    return np.array(targets, dtype=np.int32)

In [44]:
indicator_data = (
    pd.DataFrame(
        data=np.array([rsi_values, bb_values, mfi_values, cmf_values]).T,
        columns=["rsi", "bb", "mfi", "cmf"],
    )
    .dropna()
    .reset_index(drop=True)
    .astype(np.float64)
)
indicator_data

,rsi,bb,mfi,cmf
0,53.750000,0.834435,47.310164,0.118391
1,53.750000,0.979976,56.327443,0.202426
2,65.671642,1.006511,60.871871,0.235202
3,61.016949,0.910924,67.173006,0.238298
4,61.016949,2.210979,74.939408,0.378479
...,...,...,...,...
2113,57.291667,1.691918,32.142454,-0.255301
2114,68.800000,2.966376,47.706655,-0.086727
2115,70.895522,2.461380,53.031262,0.006295
2116,66.901408,1.604800,51.008568,0.002142


In [45]:
indicator_data.describe()

,rsi,bb,mfi,cmf
count,2118.000000,2118.000000,2118.000000,2118.000000
mean,49.895664,-0.024507,44.188489,0.098081
std,16.466172,1.252253,18.323338,0.243771
min,2.247191,-3.465921,1.173070,-0.772300
25%,38.160406,-1.015955,30.561603,-0.068747
50%,49.593496,-0.053811,43.494468,0.109617
75%,61.283057,0.970050,57.039061,0.263337
max,96.000000,3.330035,96.698208,0.766618


# Обработка фичей

In [46]:
scaler = MinMaxScaler()
scaler.fit(indicator_data.iloc[:, :-1])
indicator_data.iloc[:, :-1] = scaler.transform(indicator_data.iloc[:, :-1])
indicator_data

,rsi,bb,mfi,cmf
0,0.549347,0.632782,0.482984,0.118391
1,0.549347,0.654197,0.577381,0.202426
2,0.676507,0.658102,0.624954,0.235202
3,0.626859,0.644037,0.690917,0.238298
4,0.626859,0.835335,0.772219,0.378479
...,...,...,...,...
2113,0.587123,0.758957,0.324201,-0.255301
2114,0.709875,0.946489,0.487134,-0.086727
2115,0.732227,0.872181,0.542875,0.006295
2116,0.689624,0.746138,0.521700,0.002142


In [47]:
indicator_data[["rsi", "bb"]].iloc[1].values

array([0.54934684, 0.65419737])

In [11]:
indicator_data

,rsi,bb,mfi,cmf
0,0.549347,0.632782,0.482984,0.118391
1,0.549347,0.654197,0.577381,0.202426
2,0.676507,0.658102,0.624954,0.235202
3,0.626859,0.644037,0.690917,0.238298
4,0.626859,0.835335,0.772219,0.378479
...,...,...,...,...
2113,0.587123,0.758957,0.324201,-0.255301
2114,0.709875,0.946489,0.487134,-0.086727
2115,0.732227,0.872181,0.542875,0.006295
2116,0.689624,0.746138,0.521700,0.002142


In [48]:
all_data = indicator_data.copy()
all_data[df.columns] = df.iloc[n_steps:, :].values

all_data

,rsi,bb,mfi,cmf,open,high,low,close,volume
0,0.549347,0.632782,0.482984,0.118391,162.46,162.50,162.45,162.48,156850.0
1,0.549347,0.654197,0.577381,0.202426,162.48,162.50,162.48,162.49,83300.0
2,0.676507,0.658102,0.624954,0.235202,162.50,162.50,162.49,162.50,52370.0
3,0.626859,0.644037,0.690917,0.238298,162.49,162.50,162.49,162.50,28020.0
4,0.626859,0.835335,0.772219,0.378479,162.50,162.63,162.49,162.63,62910.0
...,...,...,...,...,...,...,...,...,...
2113,0.587123,0.758957,0.324201,-0.255301,163.37,163.40,163.30,163.40,171700.0
2114,0.709875,0.946489,0.487134,-0.086727,163.40,163.82,163.39,163.71,623050.0
2115,0.732227,0.872181,0.542875,0.006295,163.70,163.82,163.61,163.80,314660.0
2116,0.689624,0.746138,0.521700,0.002142,163.80,163.82,163.60,163.70,136580.0


In [49]:
# Your DataFrame `df` should contain all observations and you will have your reward function `reward_function`
# df = all_da.copy()
# df.columns = [col.lower() for col in df.columns]

steps_obs = 1
commision_bps = 0
rewards = []
Statistics = []

init_balance = 200
init_active = 10

start_price = all_data["close"].iloc[0]
init_pv = init_balance + init_active * start_price
rnn_steps = 2

MAX_BALANCE = 2 * init_pv
MAX_ACTIVE = 2 * init_pv / start_price


class CustomEnv(gym.Env):
    def __init__(self, df, init_balance, init_active, test=False):
        super(CustomEnv, self).__init__()
        self.df = df
        self.test = test
        self.seed = lambda x: 0
        self.current_step = 0
        # Assume observation space is a 1D array of features (adjust according to your data)
        self.observation_space = spaces.Box(
            low=0,
            high=1,
            shape=(rnn_steps, len(indicator_data.columns) + 2),
            dtype=np.float32,
        )
        self.balance = init_balance
        self.active = init_active
        self.init_balance = init_balance
        self.init_active = init_active
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.balance + self.active * self.current_price
        # Adjust according to your action space
        self.action_space = spaces.Box(
            low=np.array([-1]),
            high=np.array([1]),
            dtype=np.float32,
        )

    def reset(self):
        # Reset the state of the environment to an initial state
        print("reseting...")
        self.current_step = rnn_steps - 1
        if self.test:
            rewards.append([])
            Statistics.append([])

        init_state = np.append(
            self.df[indicator_data.columns]
            .iloc[self.current_step - rnn_steps + 1 : self.current_step + 1]
            .values,
            np.array(
                [
                    [self.balance / MAX_BALANCE, self.active / MAX_ACTIVE]
                    for i in range(rnn_steps)
                ]
            ),
            axis=1,
        )
        self.balance = init_balance
        self.active = init_active
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.balance + self.init_active * self.current_price
        return init_state

    def reward_function(self, action):
        action = float(action)
        next_price = self.df["close"].iloc[self.current_step + 1]
        # покупаем
        if action > 0:
            money_to_spend = self.balance * action
            self.active += money_to_spend / self.current_price
            self.balance -= money_to_spend
        # продаем
        elif action < 0:
            active_to_sell = self.active * abs(action)
            self.active -= active_to_sell
            self.balance += active_to_sell * self.current_price
        new_pv = self.active * next_price + self.balance
        reward = (new_pv - self.pv) / self.pv * 1e4

        return reward

    def step(self, action):
        # Calculate the reward

        reward = self.reward_function(action)
        self.current_step += 1
        if self.current_step < len(self.df.index) - (steps_obs + 1):
            next_state = np.append(
                self.df[indicator_data.columns]
                .iloc[self.current_step - rnn_steps + 1 : self.current_step + 1]
                .values,
                np.array(
                    [
                        [self.balance / MAX_BALANCE, self.active / MAX_ACTIVE]
                        for i in range(rnn_steps)
                    ]
                ),
                axis=1,
            )
            done = False
        else:
            # The episode is finished
            next_state = np.zeros(self.observation_space.shape)
            done = True

        # Placeholder for info, additional data can be added if needed
        info = {}
        # print(next_state)
        if self.test:
            rewards[-1].append(reward)
        self.current_price = self.df["close"].iloc[self.current_step]
        self.pv = self.active * self.current_price + self.balance

        return next_state, reward, done, info


# Create multiple environments
env = CustomEnv(df=all_data, init_balance=init_balance, init_active=init_active)
env = DummyVecEnv([lambda: env])

In [50]:
# Сохранение и загрузка модели
# model.save("./data/PPO_MultiBinScaling.zip")
# model = PPO.load("./data/PPO_MultiBinScaling.zip", env=env)

In [59]:
policy_kwargs = dict(
    activation_fn=nn.ReLU,
    net_arch=dict(pi=[20, 20], vf=[20, 20]),
    lstm_hidden_size=8,
    n_lstm_layers=1,
)

# Create the PPO model
model = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,
    device="cpu",
    policy_kwargs=policy_kwargs,
    learning_rate=0.0001,
    batch_size=128,  # размер одного батча
    n_epochs=5,  # количество повторов на каждом новом наборе батчей
    gamma=0.99,  # степень заглядывания в будущее (от 0 до 0.99999)
    n_steps=128 * 4,  # batch_size x количество батчей
)

# # Train the model
model.learn(total_timesteps=8000)  # , progress_bar=True)

# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=2)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

Using cpu device
reseting...
----------------------------
| time/              |     |
|    fps             | 189 |
|    iterations      | 1   |
|    time_elapsed    | 2   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 161           |
|    iterations           | 2             |
|    time_elapsed         | 6             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 5.2375253e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.42         |
|    explained_variance   | -0.000518     |
|    learning_rate        | 0.0001        |
|    loss                 | 15.9          |
|    n_updates            | 5             |
|    policy_gradient_loss | -1.56e-05     |
|    std                  | 1             |
|    value_loss     

In [60]:
# Reset the environment to get the initial observation
obs = env.reset()

num_iterations = 20
actions_taken = []
rewards = []

for _ in range(num_iterations):
    print(obs)
    # Choose an action randomly (replace this with your own policy)
    action, _ = model.predict(obs, deterministic=False)

    # Take a step in the environment with the predicted action
    obs, reward, done, _ = env.step(action)

    # Save the action taken
    actions_taken.append(action)
    rewards.append(reward)

    if done:
        break

reseting...
[[[0.54934686 0.63278157 0.4829838  0.11839067 0.05480053 0.44519946]
  [0.54934686 0.6541974  0.5773807  0.20242633 0.05480053 0.44519946]]]
[[[0.54934686 0.6541974  0.5773807  0.20242633 0.2044625  0.2955467 ]
  [0.67650723 0.65810204 0.6249538  0.23520164 0.2044625  0.2955467 ]]]
[[[0.67650723 0.65810204 0.6249538  0.23520164 0.3059261  0.19409558]
  [0.62685865 0.6440367  0.69091696 0.23829797 0.3059261  0.19409558]]]
[[[0.62685865 0.6440367  0.69091696 0.23829797 0.419828   0.08020771]
  [0.62685865 0.835335   0.7722191  0.37847912 0.419828   0.08020771]]]
[[[0.62685865 0.835335   0.7722191  0.37847912 0.39571083 0.10430263]
  [0.65156597 0.79856586 0.73336536 0.23354925 0.39571083 0.10430263]]]
[[[0.65156597 0.79856586 0.73336536 0.23354925 0.419798   0.08024065]
  [0.9214569  0.7635302  0.6770725  0.07029834 0.419798   0.08024065]]]
[[[0.9214569  0.7635302  0.6770725  0.07029834 0.42824516 0.07180284]
  [0.9715564  0.929557   0.78918284 0.30962658 0.42824516 0.071802

In [61]:
test_df = all_data.iloc[rnn_steps - 1 : num_iterations + (rnn_steps - 1), :].copy()
test_df["actions"] = np.array(actions_taken).T[0][0]
test_df["rewards"] = np.array(rewards)

test_df

,rsi,bb,mfi,cmf,open,high,low,close,volume,actions,rewards
1,0.549347,0.654197,0.577381,0.202426,162.48,162.50,162.48,162.49,83300.0,-0.336148,0.363775
2,0.676507,0.658102,0.624954,0.235202,162.50,162.50,162.49,162.50,52370.0,-0.343266,0.000000
3,0.626859,0.644037,0.690917,0.238298,162.49,162.50,162.49,162.50,28020.0,-0.586762,1.283364
4,0.626859,0.835335,0.772219,0.378479,162.50,162.63,162.49,162.63,62910.0,0.057445,0.256720
5,0.651566,0.798566,0.733365,0.233549,162.62,162.69,162.62,162.65,54210.0,-0.230694,0.098746
6,0.921457,0.763530,0.677073,0.070298,162.65,162.67,162.64,162.66,29610.0,-0.105156,2.915915
7,0.971556,0.929557,0.789183,0.309627,162.65,162.99,162.64,162.99,325840.0,0.137305,-1.444029
8,0.848732,0.786334,0.729584,0.291088,162.98,163.00,162.86,162.90,189210.0,0.522328,1.986480
9,0.894147,0.771601,0.749822,0.341364,162.90,163.00,162.84,162.95,236420.0,-1.000000,0.000000
10,0.779470,0.685548,0.707404,0.290688,162.92,162.95,162.85,162.87,74430.0,-0.229764,0.000000


In [62]:
fig = px.line(
    test_df, x=test_df.index, y="close", hover_data=["rsi", "bb", "actions", "rewards"]
)
fig.update_traces(mode="markers+lines")

fig.show()